In [2]:
from IPython.display import HTML, display

def print_banner(title, gradient_colors=['#FF5F6D', '#FFC371']):
  banner_html = f"""
  <div style="background: linear-gradient(to right, {gradient_colors[0]}, {gradient_colors[1]}); padding: 10px; border-radius: 5px; text-align: center;">
    <h1 style="color: white; font-family: sans-serif;">{title}</h1>
  </div>
  """
  display(HTML(banner_html))

In [3]:
def print_sub_banner(title, gradient_colors=['#4CAF50', '#8BC34A']): # Green gradient
  banner_html = f"""
  <div style="background: linear-gradient(to right, {gradient_colors[0]}, {gradient_colors[1]}); padding: 10px; border-radius: 5px; text-align: center;">
    <h2>{title}</h2>
  </div>
  """
  display(HTML(banner_html))


In [4]:
print_banner("Installation des librairies")

In [5]:
from lxml import html
import requests
import pandas as pd
import plotly.express as px
import numpy as np
import time
from tqdm import tqdm
import pymongo
import json
import folium

In [6]:
print_sub_banner("Demande de Valeurs Foncières (DFV3F)")

In [11]:
#les Apis requetés
#L'API de l'API des indicateurs de la Demande de Valeurs Foncières (DFV3F) est une API gratuite qui permet de faire d'avoir accès à un historique des prix des ventes immobilières.
BASE_URL_API = "https://apidf-preprod.cerema.fr"

In [9]:
def apidf(url_endpoint, token=None):
    HEADERS = {
        "Content-Type": "application/json",
    }
    if token:
        HEADERS["Authorization"] = "Token " + token
    response = requests.get(
        url_endpoint,
        headers=HEADERS,
    )
    if response.status_code == 200:
      return response.json()
    return None

In [12]:
# Paramétrage du endpoint
code_insee, nom = "35238", "Rennes"
url = BASE_URL_API + f"/indicateurs/dv3f/communes/annuel/{code_insee}"

# Interrogation de l'API et récupération d'un dataframe
response  = apidf(url)
indicateurs = pd.DataFrame.from_dict(response["results"])

# Edition du graphique
fig = px.bar(indicateurs,
             x='annee',
             y=['nbtrans_cod111', 'nbtrans_cod121'],
             title = f"Evolution annuelle du nombre de ventes de logements individuels à {nom}",
             labels={"annee" : "Année de mutation",
                     "value" : "Nombre de ventes",},
             )
noms={"nbtrans_cod111": "Maison individuelle",
      "nbtrans_cod121": "Appartement individuel"}
fig.update_layout(legend_title_text="Nombre de ventes")
fig.for_each_trace(lambda t: t.update(hovertemplate = t.hovertemplate.replace(t.name, noms[t.name]), name=noms[t.name]))
fig.show()

In [ ]:
# Nous centrons sur Rennes
x, y = -1.676587234535742, 48.11772222119084
delta_x, delta_y = 0.007, 0.007

# définition de la bbox
x1 = x - delta_x
y1 = y - delta_y
x2 = x + delta_x
y2 = y + delta_y

# creation de la carte
m = folium.Map(location=[y, x], zoom_start=16)

url = BASE_URL_API + f"/dvf_opendata/geomutations/?in_bbox={x1},{y1},{x2},{y2}&page_size=1000"
# ajout d'une couche geojson pour chaque page de données
while True:
    response = apidf(url)

    folium.GeoJson(response,
                   name="mutations",
                   popup=folium.GeoJsonPopup(fields=["libtypbien", "datemut", "valeurfonc", "sbati", "sterr"]),).add_to(m)
    if not response["next"]:
      break

    url = response["next"]

m